In [2]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import implicit
import json
import math
import numpy as np
from sklearn.metrics import ndcg_score

In [14]:
# %load /home/ivan/Documents/upf/inforetrieval/finalproject/information_retrieval_final_project/search-engine/tweet.py
import json

class Tweet:
    def __init__(self, id, text, user, date, url,
                 hashtags, likes, retweets, terms):
        self.id = int(id)
        self.text = text
        self.user = int(user)
        self.date = date
        self.url = url
        self.hashtags = hashtags
        self.likes = likes
        self.retweets = retweets
        self.terms = terms

    def get_terms(self):
        return self.terms

    @staticmethod
    def from_json(json_str):
        tweet_json = json.loads(json_str)
        return Tweet(
            tweet_json['ID'], tweet_json['Tweet_text'], tweet_json['UserId'],
            tweet_json['Date'], tweet_json['URL'], tweet_json['Hashtags'],
            tweet_json['Likes'], tweet_json['Number_Retweets'], tweet_json['terms']
        )

    def to_json(self):
        return {
            'ID': self.id,
            'Tweet_text': self.text,
            'UserId': self.user,
            'Date': self.date,
            'URL': self.url,
            'Hashtags': self.hashtags,
            'Likes': self.likes,
            'Number_Retweets': self.retweets,
            'terms': self.terms
        }

    def tf_idf_score(self, index):
        tweet_v = []
        tweet_tf = index.get_tf(self, self.id)
        idfs = index.get_idf(self)
        for i in range(len(query_tf)):
            tweet_v.append(tweet_tf[i] * idfs[i])
            query_v.append(query_tf[i] * idfs[i])

    def __str__(self):
        return ('%s\n %s | %s | %s | %s | %s | %s' %
                (self.text, self.user, self.date, self.hashtags, self.likes, self.retweets, self.url))

    def __hash__(self):
        return self.id # we assume ids are properly assigned to unique tweets

    def __eq__(self, other):
        if not isinstance(other, Tweet):
            return False
        return self.id == other.id # we assume ids are properly assigned to unique tweets

In [38]:
# %load /home/ivan/Documents/upf/inforetrieval/finalproject/information_retrieval_final_project/search-engine/index.py
#import scrapping.py
import math
import json
from collections import defaultdict
import numpy as np
import pickle
from heapq import heappush, heappushpop
from array import array

class Index:

    def __init__(self, repr_size=128):
        self.index= None # term --> tweet
        self.tf = None
        self.df = None
        self.tweets = {}
        self.max_fav = 0
        self.max_rt = 0
        self.repr_size = repr_size
        self.topidf = None

    def load_json_tweets(self, file):
        with open(file, 'r') as fp:
            self.create_index(fp.readlines())

    def save(self, path):
        with open(path, 'wb') as fp:
            pickle.dump(self, fp)

    @staticmethod
    def load(path):
        with open(path, 'rb') as fp:
            index = pickle.load(fp)
        return index

    def create_index(self, lines):
        """
        Implement the inverted index
        
        Argument:
        lines -- collection of tweets
        
        Returns:
        index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
        list of tweets these keys appears in (and the positions) as values.
        """

        index=defaultdict(list)
        tf = defaultdict(dict)
        df = defaultdict(int)

        for line in lines: # Remember, lines contain all tweets, each line is a tweet
            tweet_json = json.loads(line)
            tweet = Tweet(
                int(tweet_json['ID']), tweet_json['Tweet_text'], tweet_json['UserId'],
                tweet_json['Date'], tweet_json['URL'], tweet_json['Hashtags'],
                tweet_json['Likes'], tweet_json['Number_Retweets'], tweet_json['terms']
            )
            if tweet.id in self.tweets:
                continue
            self.tweets[tweet.id] = tweet
            tweet_id = tweet.id #tweet id
            terms = tweet.terms #page_title + page_text
            self.max_fav = max(self.max_fav, tweet.likes)
            self.max_rt = max(self.max_rt, tweet.retweets)
            #title = line_arr[1]            
            #titleIndex[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
            
            ## ===============================================================        
            ## create the index for the current doc and store it in termdictPage
            ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
            
            ## Example: if the curr_doc has id 1 and his text is 
            ## "web retrieval information retrieval":
            
            ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,3]], ‘information’: [1, [2]]}
            
            ## the term ‘web’ appears in document 1 in positions 0, 
            ## the term ‘retrieval’ appears in document 1 in positions 1 and 3
            ## ===============================================================
            
            termdictTweets={}

            for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
                try:
                    # if the term is already in the index for the current page (termdictPage)
                    # append the position to the corrisponding list
                    
            ## START CODE
                    termdictTweets[term][1].append(position)  
                except:
                    # Add the new term as dict key and initialize the array of positions and add the position
                    termdictTweets[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in python)
                
            #merge the current page index with the main index
            for termpage, postingpage in termdictTweets.items():
                index[termpage].append(postingpage)

            # normalize term frequencies
            # Compute the denominator to normalize term frequencies (formula 2 above)
            # norm is the same for all terms of a document.
            norm = 0
            for term, posting in termdictTweets.items():
                # posting is a list containing doc_id and the list of positions for current term in current document:
                # posting ==> [currentdoc, [list of positions]]
                # you can use it to inferr the frequency of current term.
                norm += len(posting[1]) ** 2
            norm = math.sqrt(norm)

            # calculate the tf(dividing the term frequency by the above computed norm) and df weights
            for term, posting in termdictTweets.items():
                # append the tf for current term (tf = term frequency in current doc/norm)
                tf[term][posting[0]] = np.round(len(posting[1]) / norm, 4) ## SEE formula (1) above
                # increment the document frequency of current term (number of documents containing the current term)
                df[term] += 1  # increment df for current term
            ## END CODE                    
        self.index = index
        self.tf = tf
        self.df = df
        N = len(self.tweets)
        self.topidf = []
        for term, df in self.df.items():
            if len(self.topidf) < self.repr_size:
                heappush(self.topidf, (df, term))
            else:
                heappushpop(self.topidf, (df, term))

    #Given a query (set of words) and and index return the idf of that query
    def get_idf(self, query):
        N=len(self.tweets)
        idf=[]
        query=query.get_terms()
        for term in query:
            df=self.df[term]
            idf.append(math.log(N/df))
        return idf

    #Given a query (set of words) and and index return the total tf of that query 
    def get_tf(self, query, tweet_id):
        query=query.get_terms()
        global_tf_list=[]
        term_list=[]
        for term in query:
            term_list.append(self.tf[term][tweet_id])
        return term_list

    def get_tf_idf_vector(self, tweet_id):
        vector = np.zeros(self.repr_size)
        N = len(self.tweets)
        total = 0
        for i, (df, term) in enumerate(self.topidf):
            tf = self.tf[term].get(tweet_id, 0)
            tf_idf = tf * math.log(N/df)
            vector[i] = tf_idf
        norm = np.linalg.norm(vector)
        if norm == 0:
            return vector
        return vector / norm

    def get_all_tf_idf(self):
        vectors = np.array([])
        for tweet in self.tweets:
            v = self.get_tf_idf_vector(tweet)
            if isinstance(v, np.ndarray):
                vectors = np.vstack((vectors, v))
            if len(vectors) % 1000 == 0:
                print(len(vectors))
        return vectors

    #Given a query (set of words) return all the tweets containing the query
    def get_tweets(self, query):
        query=query.get_terms()
        list_tweets=set()
        for term in query:
            ## START DODE
            try:
                # store in termDocs the ids of the docs that contain "term"
                termTweets=[posting[0] for posting in self.index[term]]
                # docs = docs Union termDocs
                if len(list_tweets) == 0:
                    list_tweets = list_tweets.union(termTweets)
                else:
                    list_tweets = list_tweets.intersection(termTweets)
            except:
                #term is not in index
                pass
        list_tweets=list(list_tweets)

        tweets_query = [self.tweets[tid] for tid in list_tweets] #asssuming index of tweets is id
        #list of tweets ids
        return tweets_query

    #Given a query (set of words) return the tweets number of tweets containing the query
    def get_num_tweets(self,query):

        num_tweets=len(self.get_tweets(query))

        return num_tweets

    def get_max_fav(self):
        return self.max_fav

    def get_max_rt(self):
        return self.max_rt


In [12]:
def cosine_similarity(a, b, norm_a=True, norm_b=True):
    total = 0
    sum_a = 0 if norm_a else 1.0
    sum_b = 0 if norm_b else 1.0
    for x, y in zip(a, b):
        total += x * y
        if norm_a:
            sum_a += x * x
        if norm_b:
            sum_b += y * y
    if norm_a:
        sum_a = math.sqrt(sum_a)
    if norm_b:
        sum_b = math.sqrt(sum_b)
    return total / (sum_a * sum_b)

In [17]:
# %load /home/ivan/Documents/upf/inforetrieval/finalproject/information_retrieval_final_project/search-engine/user.py
import json
import numpy as np

class User:
    def __init__(self, id, username, mentions):
        self.id = int(id)
        self.username = username
        self.mentions = mentions

    def add_mention(self, uid):
        if uid not in self.mentions:
            self.mentions[uid] = 0
        self.mentions[uid] += 1

    @staticmethod
    def from_dict(data):
        mentions = {int(user): int(mention) for user, mention in data['MENTIONS'].items()}
        return User(int(data['USERID']), data['USERNAME'], mentions)

    @staticmethod
    def from_json(json_str):
        user_data = json.loads(json_str)
        return User.from_dict(user_data)

    def to_json(self):
        return {
            'USERID' : self.id,
            'USERNAME' : self.username,
            'MENTIONS' : self.mentions}

class UserTweets:
    def __init__(self, user, tweets):
        self.user = user
        self.tweets = tweets

    def add_tweet(self, tweet):
        self.tweets.append(tweet)

    def get_vector(self, index):
        return np.mean([index.get_tf_idf_vector(t.id) for t in self.tweets], axis=0)

def load_users(file):
    with open(str(file), 'r') as fp:
        for line in fp.readlines():
            file_users = json.loads(line)
    return {int(uid):User.from_dict(user_dict)
                for uid, user_dict in file_users.items()}

def get_user_tweets(users, index):
    user_tweets = {}
    for tweet in index.tweets.values():
        if tweet.user in users:
            user = users[tweet.user]
            if user.id not in user_tweets:
                user_tweets[user.id] = UserTweets(user, [])
            user_tweets[user.id].add_tweet(tweet)
    return user_tweets

def user_relevance(user1: UserTweets, user2: UserTweets):
    return cosine_similarity(user1.get_vector(), user2.get_vector())

In [7]:
def create_user_graph(users):
    graph = nx.Graph()
    for user in users.values():
        graph.add_node(user.id)
    for user in users.values():
        for mention in user.mentions.keys():
            graph.add_edge(user.id, mention)
    return graph

In [78]:
users = load_users('../res/merge_users.json')

In [9]:
graph = create_user_graph(users)

In [79]:
index = Index()
index.load_json_tweets('../res/merge_tweets_wusers.json')

In [93]:
print('Number of nodes:', len(graph.nodes()))
print('Number of edges:', len(graph.edges()))

Number of nodes: 161469
Number of edges: 114622


In [22]:
def split_graph(graph, p_test):
    E_test = int(len(graph.edges) * p_test)
    test_edge_list_split = random.sample(graph.edges(), E_test)
    test_edge_list = list(test_edge_list_split)
    # Remove some edges
    training_graph = graph.copy()
    training_graph.remove_edges_from(test_edge_list_split)
    test_graph = nx.Graph()
    test_graph.add_edges_from(test_edge_list)
    return training_graph, test_graph

In [56]:
def sample_graph(graph, max_nodes):
    if len(graph.nodes()) <= max_nodes:
        return graph
    sample_nodes = random.sample(graph.nodes(), max_nodes)
    g = nx.Graph()
    for node in sample_nodes:
        g.add_node(node)
    for u, v in graph.edges():
        if u in g.nodes() and v in g.nodes():
            g.add_edge(u, v)
    return g

In [57]:
small_graph = sample_graph(graph, 50000)

In [ ]:
user_tweets = get_user_tweets(users, index)

In [58]:
print('Number of nodes:', len(small_graph.nodes()))
print('Number of edges:', len(small_graph.edges()))

Number of nodes: 50000
Number of edges: 13427


In [59]:
train, test = split_graph(small_graph, 0.20)

In [60]:
print('Number of nodes in train:', len(train.nodes()))
print('Number of nodes in test:', len(test.nodes()))
print('Number of edges in train:', len(train.edges()))
print('Number of edges in test:', len(test.edges()))

Number of nodes in train: 50000
Number of nodes in test: 3397
Number of edges in train: 10742
Number of edges in test: 2685


In [61]:
def friends_of_friends(start, graph):
    friends1 = set(graph.neighbors(start))
    friends = set()
    for friend in friends1:
        friends = friends.union(set(graph.neighbors(friend)))
    friends = friends.difference({start})
    friends = friends.difference(friends1)
    return friends

In [77]:
users[2596370295].username

'MichaelTheStdnt'

In [76]:
for user in friends_of_friends(2596370295, small_graph):
    print(users[user].username)

jacobinmag
shantilly_t
ASNinaWrites
rabbitven0m
AntifaVio
TeenVogue
prolpo
cit_uprising
JamaalBowmanNY
knjatz
SmythLr
yimmygee
RaceJustice
reinedeloup
MrErnestOwens
shannondrewthis
QasimRashid


In [ ]:
def ndcg_graph(recommendations, graph, K=10):
    scores = []
    true_rel = []
    for user in recommendations:
        score = [sc for u, sc in recommendations[user][:K]]
        if len(score) == 0:
            continue
        if len(score) < K:
            score += [0] * (K - len(score))
        t_rel = [int((user, u) in graph.edges()) for u, sc in recommendations[user][:K]]
        if len(t_rel) < K:
            t_rel += [0] * (K - len(t_rel))
        scores.append(score)
        true_rel.append(t_rel)
    return ndcg_score(np.array(true_rel), np.array(scores))

In [72]:
def pagerank(g_train, g_test, graph, K=10):
    recommendations = {}
    N_test = len(g_test.nodes())
    pr = nx.pagerank(g_train,personalization={node: 1/N_test for node in g_test.nodes()})
    recommendations = {}
    for node in dict(pr).keys():
        friends = friends_of_friends(node, graph)
        node_scores = [(n, pr[node]) for n in friends]
        node_scores.sort(key=lambda x: x[1], reverse=True)
        recommendations[node] = node_scores[:K]
        recommendations[node] = [(u, score/recommendations[node][0][1]) for u, score in recommendations[node]]
    return recommendations

In [73]:
pager = pagerank(train, test, small_graph)

In [74]:
for user, _ in pager[2596370295]:
    print(users[user].username)

jacobinmag
shantilly_t
ASNinaWrites
rabbitven0m
AntifaVio
TeenVogue
prolpo
cit_uprising
JamaalBowmanNY
knjatz


In [69]:
def adamic_adar(g_train, g_test, graph, K=10):
    edges = set()
    for node in g_test.nodes():
        edges = edges.union({(node, friend) for friend in friends_of_friends(node, graph)})
    preds = nx.adamic_adar_index(g_train, list(edges))
    recommendation = {}
    for u, v, p in preds:
        if u not in recommendation:
            recommendation[u] = []
        recommendation[u].append((v, p))
    for u, rank in recommendation.items():
        rank.sort(key=lambda x: x[1], reverse=True)
        recommendation[u] = rank[:K]
    return recommendation

In [70]:
adamic_result = adamic_adar(train, test, small_graph)

In [71]:
for user, _ in adamic_result[2596370295]:
    print(users[user].username)

rabbitven0m
reinedeloup
JamaalBowmanNY
RaceJustice
cit_uprising
prolpo
MrErnestOwens
yimmygee
shannondrewthis
knjatz


In [54]:
def als(g_train, g_test, graph, K=10):
    als = implicit.als.AlternatingLeastSquares()
    als.fit(nx.adjacency_matrix(g_train, nodelist=graph.nodes()))
    index = {node:i for i, node in enumerate(graph.nodes())}
    A = nx.adjacency_matrix(graph)
    recommendations = {}
    for node in g_test.nodes():
        ff = [index[friend] for friend in friends_of_friends(node, graph)]
        if len(ff) == 0:
            recommendations[node] = []
            continue
        rank = als.rank_items(index[node], A, ff)
        recommendations[node] = [(list(graph.nodes())[i], score) for i, score in rank[:K]]
    return recommendations
    

In [62]:
als_result = als(train, test, small_graph)

In [64]:
for user, _ in als_result[2596370295]:
    print(users[user].username)

rabbitven0m
SmythLr
shantilly_t
RaceJustice
shannondrewthis
MrErnestOwens
reinedeloup
prolpo
yimmygee
knjatz


In [47]:
def custom_relevance(g_train, g_test, graph, user_tweets, index, K=10):
    recommendations = {}
    for user in g_test.nodes():
        recommendations[user] = []
        if user not in user_tweets:
            continue
        vector = user_tweets[user].get_vector(index)
        for friend in friends_of_friends(user, g_train):
            if friend in user_tweets:
                friend_vector = user_tweets[friend].get_vector(index)
                recommendations[user].append((friend, cosine_similarity(vector, friend_vector)))
        recommendations[user].sort(key=lambda x: x[1], reverse=True)
        recommendations[user] = recommendations[user][:K]
    return recommendations

In [66]:
custom = custom_relevance(train, test, small_graph, user_tweets, index)

/home/ivan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [67]:
for user, _ in custom[2596370295]:
    print(users[user].username)

reinedeloup
ASNinaWrites
jacobinmag
TeenVogue
JamaalBowmanNY
prolpo
yimmygee
rabbitven0m
MrErnestOwens
AntifaVio
